In [1]:
import pandas as pd
import tensorflow as tf 

train_data = pd.read_csv('/code/mnist/train.csv')
test_data = pd.read_csv('/code/mnist/test.csv')

X = train_data.drop('label', axis=1)
Y = train_data['label']

2023-07-29 02:51:47.409615: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-29 02:51:47.470975: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-29 02:51:47.897409: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-29 02:51:47.900036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 02:51:49.118228: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
X = X.astype('float32') / 255
test_data = test_data.astype('float32') / 255

Y = tf.keras.utils.to_categorical(Y, 10)

In [3]:
train_x = X[:round(X.shape[0] * 0.8)]
check_x = X[round(X.shape[0] * 0.8):]

train_y = Y[:round(Y.shape[0] * 0.8)]
check_y = Y[round(Y.shape[0] * 0.8):]

In [4]:
# from tensorflow.keras.layers import Dense, Dropout

# inputs = tf.keras.Input(shape=(784,))
# x_1 = Dense(256, activation='relu')(inputs)
# x_2 = Dropout(0.2)(x_1)
# outputs = Dense(10, activation='softmax')(x_2)

# model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [5]:
train_x = train_x.values.reshape(-1, 28, 28, 1)
check_x = check_x.values.reshape(-1, 28, 28, 1)

test_data = test_data.values.reshape(-1, 28, 28, 1)

In [6]:
train_x.shape

(33600, 28, 28, 1)

In [7]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten


inputs = tf.keras.Input(shape=(28, 28, 1))
x_1 = Conv2D(32, kernel_size=(5, 5), strides=1, activation='relu', padding='same')(inputs)
x_2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x_1)
x_3 = Flatten()(x_2)
x_4 = Dropout(0.4)(x_3)
outputs = Dense(10, activation='softmax')(x_4)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

2023-07-29 02:51:54.202543: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [8]:
LEARNING_RATE = 0.001
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=[tf.keras.metrics.categorical_accuracy],
)

In [9]:
train_y.shape

(33600, 10)

In [10]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=2, 
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/code/mnist/data/temp/mnist_sequential_{epoch:03d}_{val_loss:.4f}.h5',
        save_best_only=True
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir='/code/mnist/logs',
        histogram_freq=1
    ),
]

BATCH_SIZE = 128
EPOCHS = 20
VALIDATION_SPLIT = 0.2

history = model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, callbacks=callbacks)
model.save('/code/mnist/mnist_model.h5')

Epoch 1/20
210/210 [==============================] - 4s 16ms/step - loss: 0.4887 - categorical_accuracy: 0.8669 - val_loss: 0.2239 - val_categorical_accuracy: 0.9375
Epoch 2/20
  1/210 [..............................] - ETA: 3s - loss: 0.2403 - categorical_accuracy: 0.9297

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


210/210 [==============================] - 3s 15ms/step - loss: 0.1856 - categorical_accuracy: 0.9472 - val_loss: 0.1430 - val_categorical_accuracy: 0.9597
Epoch 3/20
210/210 [==============================] - 3s 15ms/step - loss: 0.1316 - categorical_accuracy: 0.9614 - val_loss: 0.1154 - val_categorical_accuracy: 0.9662
Epoch 4/20
210/210 [==============================] - 3s 16ms/step - loss: 0.1053 - categorical_accuracy: 0.9697 - val_loss: 0.0948 - val_categorical_accuracy: 0.9732
Epoch 5/20
210/210 [==============================] - 3s 15ms/step - loss: 0.0880 - categorical_accuracy: 0.9731 - val_loss: 0.0841 - val_categorical_accuracy: 0.9762
Epoch 6/20
210/210 [==============================] - 4s 18ms/step - loss: 0.0798 - categorical_accuracy: 0.9751 - val_loss: 0.0776 - val_categorical_accuracy: 0.9774
Epoch 7/20
210/210 [==============================] - 3s 15ms/step - loss: 0.0712 - categorical_accuracy: 0.9776 - val_loss: 0.0715 - val_categorical_accuracy: 0.9792
Epoch 8/2

In [11]:
model.summary()

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dropout (Dropout)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 10)                62730     
                                                       

In [12]:
model.evaluate(check_x, check_y)

263/263 [==============================] - 1s 2ms/step - loss: 0.0611 - categorical_accuracy: 0.9801


[0.061085257679224014, 0.9801190495491028]

- 0.9811905026435852
- 0.98416668176651 maxpooling
- 0.9783333539962769 karnel(5,5) => (3,3) & padding 'same'
- 0.9825000166893005 karnel => (3,3)
- 0.9840475916862488 drop 0.2 => 0.4

In [13]:
submit = pd.DataFrame()
submit['ImageId'] = [i for i in range (1,test_data.shape[0]+1)]
submit['Label'] = model.predict(test_data).argmax(axis=1)

submit.to_csv('/code/mnist/submit.csv',index=False)

875/875 [==============================] - 2s 2ms/step
